In [1]:
import gym

import torch
import torch.nn as nn

from models.ddpg import DDPG

In [2]:
class Actor(nn.Module):
    def __init__(self, n_sts: int, n_acts: int, dim:int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(n_sts, dim)
        self.fc2 = nn.Linear(dim, n_acts)
        
        self.relu1 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = torch.tanh(self.fc2(x)) * 2
        
        return x

In [3]:
class Critic(nn.Module):
    def __init__(self, n_sts: int, n_acts: int, dim: int):
        super(Critic, self).__init__()
                
        self.fc1 = nn.Linear(n_sts + n_acts, dim)
        self.fc2 = nn.Linear(dim, 1)
        
        self.relu1 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(torch.cat([x, a], 1)))
        x = self.fc2(x)
        
        return x

In [4]:
cri_lr = 1e-3
act_lr = 5e-4

tau = 5e-3

n_sts = 3
n_acts = 1

dim = 256

n_epis = 500
n_epochs = 200
n_rollout = 10

act_range = (-2, 2)

In [5]:
env = gym.make('Pendulum-v1', render_mode='human')

In [6]:
act = Actor(n_sts, n_acts, dim)
act_opt = torch.optim.Adam(act.parameters(), lr=act_lr)

cri = Critic(n_sts, n_acts, dim)
cri_opt = torch.optim.Adam(cri.parameters(), lr=cri_lr)

In [7]:
ddpg = DDPG(env, n_acts, act, act_opt, cri, cri_opt, act_noise = 0.3, act_range=act_range)

In [8]:
ddpg.train(n_epis, n_epochs, n_rollout)

c:\Users\esder\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


step: 210, score: -61.83646164879857, n_buffer: 210
step: 410, score: -83.69759130678888, n_buffer: 410
step: 610, score: -82.69868950526609, n_buffer: 610
step: 810, score: -84.07256207329831, n_buffer: 810
step: 1010, score: -83.04005912302848, n_buffer: 1010
step: 1210, score: -83.68563765907781, n_buffer: 1210
step: 1410, score: -82.15144654607087, n_buffer: 1410
step: 1610, score: -84.26065908380487, n_buffer: 1610
step: 1810, score: -79.63306144798506, n_buffer: 1810
step: 2210, score: -82.35064201628835, n_buffer: 2210
step: 2410, score: -90.3913659187256, n_buffer: 2410
step: 2610, score: -87.21846455886937, n_buffer: 2610
step: 2810, score: -77.4668098470172, n_buffer: 2810
step: 3010, score: -76.38196673078389, n_buffer: 3010
step: 3210, score: -76.5729171142988, n_buffer: 3210
step: 3410, score: -76.37463161676256, n_buffer: 3410
step: 3610, score: -76.62613823621702, n_buffer: 3610
step: 3810, score: -76.48818542719066, n_buffer: 3810
step: 4210, score: -73.58899221674714, 

KeyboardInterrupt: 

In [9]:
env.close()